In [25]:
from typing import Optional, List
class LinkedListNode:
    def __init__(self, key=None, value=0, next=None, prev=None):
        self.key = key
        self.value = value
        self.next = next
        self.prev = prev
    
    def __repr__(self):
        return str(self.value)

class LinkedList:
    def __init__(self, iterable: List[int] = []):
        self.head = None
        self.tail = None
        self.size = 0

        if iterable:
            for value in iterable:
                self.append(LinkedListNode(value))
                self.size += 1
        
    def __bool__(self):
        return self.size != 0

    def __repr__(self):
        nodes = []
        current_node = self.head
        while current_node:
            nodes.append(str(current_node.value))
            current_node = current_node.next
        return ' <-> '.join(nodes)

    def append(self, new_node: LinkedListNode|int):
        if type(new_node) is int:
            new_node = LinkedListNode(new_node)
            
        if self.head is None:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.size += 1

    def remove(self, node: LinkedListNode):
        """
        Removes a given node from the linked list
        """

        if not node:
            return 
        
        if node == self.head:
            self.head = self.head.next
            self.head.prev = None
        elif node == self.tail:
            self.tail = self.tail.prev
            self.tail.next = None
        else:
            node.prev.next = node.next
            node.next.prev = node.prev
    
        self.size -= 1
        return
    

    def find_key(self, key: int) -> Optional[LinkedListNode]:
        """
        Returns a pointer to the first node with the given value
        """

        current_node = self.head
        while current_node:
            if current_node.key == key:
                return current_node
            current_node = current_node.next

        return None

In [51]:
class ClosedAddressingHashTable:

    """
    Closed Addressing Hash Table implementation from scratch.
    Closed addressing is a technique for resolving collisions in hash tables that involves
    using a fixed-size array to store the hash table entries and chaining to resolve collisions.

    Chaining:
        - Each slot in the hash table contains some data structure of items that have the same hash value.
            - Today I will use a linked list. Dynamic arrays, or BSTs can also be used.
        - When an item is inserted into the hash table, it is added to the end of the linked list along with the key. This way we can look for it properly later.
            
    Dynamic resizing:
        - When the load factor exceeds the max load factor, the hash table is resized by doubling its capacity.


    """
    def __init__(self, capacity: int = 32, load_factor: float = 0.75):
        self.size = 0
        self.capacity = capacity
        self.table = [LinkedList() for _ in range(capacity)]
        self.current_load_factor = 0
        self.max_load_factor = load_factor
   
    def __contains__(self, key) -> bool:
        table_idx = hash(key) % self.capacity
        ll = self.table[table_idx]
        result = ll.find_key(key)
        return result is not None
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, key):
        return self.get(key)
    
    def __setitem__(self, key, value):
        self.set(key, value)

    def __repr__(self):
        result = ''
        for ll in self.table:
            if ll:
                result += f"{ll}\n"
        return result
    
    """Helper methods"""

    def _check_load_factor(self):
        if self.current_load_factor >= self.max_load_factor:
            self._resize(self.capacity * 2)
        self.current_load_factor = self.size/self.capacity
        return self.current_load_factor
        
    
    def _resize(self, new_capacity: int):
        number_of_linkedlists_to_add = new_capacity - self.capacity
        self.capacity = new_capacity
        self.table += [LinkedList() for _ in range(number_of_linkedlists_to_add)]

    """Functionality"""

    def set(self, key, value):
        # Check if the key is in the hashtable
        table_idx = hash(key) % self.capacity
        ll = self.table[table_idx]
        node = ll.find_key(key)
        # If the key is not found, append new node
        if node is None:
            self._check_load_factor()
            node = LinkedListNode(key, value)
            ll.append(node)
            self.size += 1
        # If it is, update the value
        else:
            node.value = value

    def remove(self, key):
        table_idx = hash(key) % self.capacity
        ll = self.table[table_idx]
        node = ll.find_key(key)
        if node is None:
            raise KeyError(f"{key} not found")
        else:
            ll.remove(node)
            self.size -= 1
        
    def get(self, key):
        table_idx = hash(key) % self.capacity
        ll = self.table[table_idx]
        node = ll.find_key(key)

        if node is None:
            raise KeyError(f"{key} not found")
        else:
            return node.value

d = ClosedAddressingHashTable(4)
d[1] = 1
d[2] = 2
d[33] = 33
d[2] = 2.5
d[3] = 3
d[4] = 4
print(d)
print(1 in d)
print(32 in d)
print(d.size, d.current_load_factor, d.capacity)
        

4
1 <-> 33
2.5
3

True
False
5 0.5 8


In [ ]:
class OpenAddressingHashTable:
    """
    Open Addressing Hash Table implementation from scratch.
    Open addressing also known as probing is a technique for resolving collisions in hash tables.
    There are two main types of open addressing: linear probing and quadratic probing.

    Linear probing:
        - Search for the next available slot in the hash table.
        - If the slot is not available, move to the next slot.
        - If all slots are occupied, rehash the key.

    Quadratic probing:
        - Search for the next available slot in the hash table.
        - If the slot is not available, move to the next slot.
        - If all slots are occupied, rehash the key.
    """

    def __init__(self, capacity: int = 32, probe_type: str = 'linear', load_factor: float = 0.75):
        self.size = capacity
        self.capacity = 0 
        self.table = [None] * capacity
        self.current_load_factor = 0
        self.max_load_factor = load_factor

        probe_types = {
            'linear': self._linear_probe,
            'quadratic': self._quadratic_probe,
            'double': self._double_hash_probe
        }
        
        if probe_type not in probe_types:
            raise ValueError(f"Invalid probe type: {probe_type}. Valid probe types are: {list(probe_types.keys())}")
        else:
            self.probe_function = probe_types[probe_type]

   
    def __contains__(self, key) -> bool:
        table_idx = hash(key) % self.capacity
        ll = self.table[table_idx]
        result = ll.find_key(key)
        return result is not None
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, key):
        return self.get(key)
    
    def __setitem__(self, key, value):
        self.set(key, value)

    def __repr__(self):
        result = ''
        for item in self.table:
            if item is not None:
                result += f"{item}\n"
        return result

    """Helper methods"""

    def _linear_probe(self, key):
        table_idx = hash(key)
        while self.table[table_idx] is not None:
            table_idx = (table_idx + 1) % self.capacity
        return table_idx
    
    def _quadratic_probe(self, key):
        table_idx = hash(key)
        new_table_idx = table_idx
        probe_count = 0
        while self.table[new_table_idx] is not None:
            new_table_idx = (table_idx + probe_count**2) % self.capacity
            probe_count += 1
        return new_table_idx
    
    def _double_hash_probe(self, key):
        table_idx = hash(key)
        new_table_idx = table_idx
        probe_count = 0
        while self.table[new_table_idx] is not None:
            new_table_idx = (table_idx + probe_count*(hash(key) % self.capacity)) % self.capacity
            probe_count += 1
        return new_table_idx  

    def _check_load_factor(self):
        if self.current_load_factor >= self.max_load_factor:
            self._resize(self.capacity * 2)
        self.current_load_factor = self.size/self.capacity
        return self.current_load_factor 
    
    def _resize(self, new_capacity: int):
        number_of_linkedlists_to_add = new_capacity - self.capacity
        self.capacity = new_capacity
        self.table += [LinkedList() for _ in range(number_of_linkedlists_to_add)]

    """Functionality"""

    def set(self, key, value):
        pass

    def remove(self, key):
        pass
        
    def get(self, key):
        pass

d = OpenAddressingHashTable(4)
d[1] = 1
d[2] = 2
d[33] = 33
d[2] = 2.5
d[3] = 3
d[4] = 4
print(d)
print(1 in d)
print(32 in d)
print(d.size, d.current_load_factor, d.capacity)
        
